# Error Correction in Reads

## Problem

Sequencers often produce erros in their reads, with the most common error being point mutations. Becasue of this, it is important to correct read errors before we attempt to assemble a genome. In this problem, we are given a collection of up to 1000 reads of equal length (at most 50 bp) in FASTA format. Some of these reads were generated with a single-nucleotide error. Each read in the dataset was either correctly sequenced and appears in the dataset at least twice (possibly as a reverse complement) or was incorrectly sequenced and appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly one correct read in the dataset (or its reverse complement). Return a list of all corrections in the form "[old read]->[new read]". (Each correction must be a single symbol substitution, and you may return the corrections in any order.)

In [1]:
# Input: Reads of the same length 
# Output: Map of old reads to new corrected reads
# Some reads will be correct, others will be incorrect 
# the CORRECT reads will appear  AT LEAST TWICE (maybe in reverse compliment of a read)
# the INCORRECT reads will only appear once 
# INCORRECT READS will have a Hamming Distance of 1 with one other read in the dataset

# Algorithm 
# Loop through all reads
# Count how many times a read appears in the dataset
# Loop though the reverse compliments of the reads
# If reverse compliment matches any read that has been already accounted for, increment counter
# Seperate Reads by frequency, one list containing reads that appeared only once (list one)
# and one list containing reads that appeared twice or more (list two)
# Go through reads in list two and calculate the hamming distance between that read and 
# all reads in list one 
# If hamming distance == 1; match found
# If no read in list 1 returns a hamming distance == 1, repeat hamming distnace calculations 
# but this time with the reverse compliment of read 2
# If hamming distance == 1; match found
# Return read_with_error -> correct_read

In [2]:
# Import Necessary Packages
from Bio.Seq import Seq
from Bio import SeqIO
from io import StringIO
from collections import Counter

In [3]:
# Function that will calculate the Hamming Distance of two sequences
def calc_hd(seq_1, seq_2):
    hamming_distance = 0
    
    if len(seq_1) != len(seq_2):
        print("Sequences not equal in length")
        return -1
    
    for nt_1, nt_2 in zip(seq_1, seq_2):
        if nt_1!=nt_2:
            hamming_distance+=1
            
    return hamming_distance

## Sample Run

In [4]:
fasta_string = '''
>Rosalind_52
TCATC
>Rosalind_44
TTCAT
>Rosalind_68
TCATC
>Rosalind_28
TGAAA
>Rosalind_95
GAGGA
>Rosalind_66
TTTCA
>Rosalind_33
ATCAA
>Rosalind_21
TTGAT
>Rosalind_18
TTTCC
'''
f = StringIO(fasta_string)
print(f.read())


>Rosalind_52
TCATC
>Rosalind_44
TTCAT
>Rosalind_68
TCATC
>Rosalind_28
TGAAA
>Rosalind_95
GAGGA
>Rosalind_66
TTTCA
>Rosalind_33
ATCAA
>Rosalind_21
TTGAT
>Rosalind_18
TTTCC



In [5]:
print(f.seek(0))
reads = [str(record.seq) for record in SeqIO.parse(f, 'fasta')]
print(reads)

0
['TCATC', 'TTCAT', 'TCATC', 'TGAAA', 'GAGGA', 'TTTCA', 'ATCAA', 'TTGAT', 'TTTCC']


In [6]:
read_counter = Counter()

for read in reads:
    read_counter[read]+=1
    
for read in reads:
    reverse_comp = str(Seq(read).reverse_complement())
    if reverse_comp in read_counter:
        read_counter[read] += 1
        
    
print(read_counter)

Counter({'TCATC': 2, 'TGAAA': 2, 'TTTCA': 2, 'ATCAA': 2, 'TTGAT': 2, 'TTCAT': 1, 'GAGGA': 1, 'TTTCC': 1})


In [7]:
n = 0
for seq in read_counter:
    if read_counter[seq] == 1:
        n+=1
print(n)

3


In [8]:
# Correct reads
reads_list_1 = []
# Incorrect reads
reads_list_2 = []

for read in read_counter:
    if read_counter[read] >= 2:
        reads_list_1.append(read)
    else:
        reads_list_2.append(read)

print(reads_list_1, reads_list_2)

['TCATC', 'TGAAA', 'TTTCA', 'ATCAA', 'TTGAT'] ['TTCAT', 'GAGGA', 'TTTCC']


In [9]:
correction_dictionary = {}

for incorrect_read in reads_list_2:
    for correct_read in reads_list_1:
        hd = calc_hd(incorrect_read, correct_read)
        if hd == 1:
            correction_dictionary[incorrect_read] = correct_read
            reads_list_2.remove(incorrect_read)
            break

for incorrect_read in reads_list_2:
    for correct_read in reads_list_1:
        reverse_comp = str(Seq(correct_read).reverse_complement())
        hd = calc_hd(incorrect_read, reverse_comp)
        if hd == 1:
            correction_dictionary[incorrect_read] = reverse_comp
            reads_list_2.remove(incorrect_read)
            break

In [10]:
for key, val in correction_dictionary.items():
    print(f'{key}->{val}')

TTCAT->TTGAT
TTTCC->TTTCA
GAGGA->GATGA


In [11]:
filename = 'input_files/rosalind_corr.txt'

reads = [str(record.seq) for record in SeqIO.parse(filename, 'fasta')] 

read_counter = Counter()

for read in reads:
    read_counter[read]+=1
    
for read in reads:
    reverse_comp = str(Seq(read).reverse_complement())
    if reverse_comp in read_counter:
        read_counter[read] += 1
        
# Correct reads
reads_list_1 = []
# Incorrect reads
reads_list_2 = []

for read in read_counter:
    if read_counter[read] >= 2:
        reads_list_1.append(read)
    else:
        reads_list_2.append(read)

correction_dictionary = {}

for incorrect_read in reads_list_2:
    for correct_read in reads_list_1:
        hd = calc_hd(incorrect_read, correct_read)
        if hd == 1:
            correction_dictionary[incorrect_read] = correct_read
            reads_list_2.remove(incorrect_read)
            break

for incorrect_read in reads_list_2:
    for correct_read in reads_list_1:
        reverse_comp = str(Seq(correct_read).reverse_complement())
        hd = calc_hd(incorrect_read, reverse_comp)
        if hd == 1:
            correction_dictionary[incorrect_read] = reverse_comp
            break

with open('output_files/read_corrections.txt', 'a') as f:
    for key, val in correction_dictionary.items():
        f.write(f'{key}->{val}')
        f.write('\n')

In [12]:
with open('output_files/read_corrections.txt', 'r') as f:
    print(f.read())

...

